In [1]:
import sys
sys.path.append("..")

from qiskit.providers.fake_provider import Fake27QPulseV1, Fake5QV1, Fake20QV1
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from src.state_of_the_art.sabre import run_sabre
from src.state_of_the_art.cirq import run_cirq,from_backend_to_edges
from src.state_of_the_art.pytket import run_pyket
from src.state_of_the_art.qmap import run_qmap
from src.isl_routing.mapping.routing import POLY_QMAP
from src.isl_routing.utils.isl_data_loader import *
from src.isl_routing.utils.circuit_utils import *
from src.visiualisation.plots import *
from time import time


In [2]:
edges = FakeSherbrooke().configuration().coupling_map

In [3]:
file_path = f"../benchmarks/polyhedral/queko-bss-54qbt/54QBT_500CYC_QSE_0.json"
data = json_file_to_isl(file_path)

In [4]:
poly_mapper = POLY_QMAP(edges, data)

In [5]:
poly_mapper.run(heuristic_method="closure",initial_mapping_method="trivial",verbose=1)
poly_mapper.results[0]

dag generation time: 40.32067632675171


excution time: 58.60330772399902


{'swap_count': 6631, 'circuit_depth': 3074}

In [2]:
import csv

edges = FakeSherbrooke().configuration().coupling_map

def run_single_file(file_path,depth):

    data = json_file_to_isl(file_path)
    
    qmap_trivial = run_qmap(data, edges,initial_mapping="trivial")
    qmap_default = run_qmap(data, edges)
    
    swaps_trivial = qmap_trivial["swaps"]
    depth_trivial = qmap_trivial["depth"]
    
    swaps_default = qmap_default["swaps"]
    depth_default = qmap_default["depth"]
    
    return file_path,depth,swaps_trivial,depth_trivial,swaps_default,depth_default
    
qpt = 16
csv_file = f"../experiment_results/queko-bss-{qpt}qbt/qmap.csv"
os.makedirs(os.path.dirname(csv_file), exist_ok=True)

with open(csv_file, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["file_path", "depth", "swaps_trivial","depth_trivial", "swaps_default","depth_default"])
    
    for i in range(100,1000,100):
        for j in range(10):
            try:
                file_path = f"../benchmarks/polyhedral/queko-bss-{qpt}qbt/{qpt}QBT_{i}CYC_QSE_{j}.json"
                result = run_single_file(file_path,i)
                writer.writerow(result)
                f.flush()
            except:
                pass